In [18]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

In [19]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
sns.set_style('whitegrid')
from os import listdir
from os.path import isfile, join
from pandas_datareader import data
import yfinance as yf
import math
import pmdarima as pm

In [34]:
AAPL_df = pd.read_csv('collected_df.csv')
AAPL_df

,Unnamed: 0,Date,Close,^DJI_Close,^IXIC_Close,^GSPC_Close,^RLG_Close,^NYA_Close,DX-Y.NYB_Close,EURUSD=X_Close,...,GACDFSA066MSFRBPHI,GACDISA066MSFRBNY,BACTSAMFRBDAL,IR,IQ,PPIACO,CPIAUCSL,CPILFESL,MICH,CSCICP03USM665S
0,0,2015-01-02,27.332500,17832.990234,4726.810059,2058.199951,960.280029,10830.919922,91.080002,1.208941,...,13.3,12.5,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074
1,1,2015-01-05,26.562500,17501.650391,4652.569824,2020.579956,943.780029,10607.679688,91.379997,1.194643,...,13.3,12.5,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074
2,2,2015-01-06,26.565001,17371.640625,4592.740234,2002.609985,935.020020,10514.870117,91.500000,1.193902,...,13.3,12.5,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074
3,3,2015-01-07,26.937500,17584.519531,4650.470215,2025.900024,946.880005,10627.129883,91.889999,1.187536,...,13.3,12.5,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074
4,4,2015-01-08,27.972500,17907.869141,4736.189941,2062.139893,964.159973,10800.540039,92.370003,1.183600,...,13.3,12.5,-4.5,126.0,126.1,192.000,234.747,239.811,2.5,101.143074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,2002,2022-12-23,131.860001,33203.929688,10497.860352,3844.820068,2165.620117,15188.450195,104.309998,1.060333,...,-13.7,-11.2,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411
2003,2003,2022-12-27,130.029999,33241.558594,10353.230469,3829.250000,2144.899902,15218.259766,104.180000,1.064022,...,-13.7,-11.2,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411
2004,2004,2022-12-28,126.040001,32875.710938,10213.290039,3783.219971,2117.199951,15037.320312,104.459999,1.063717,...,-13.7,-11.2,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411
2005,2005,2022-12-29,129.610001,33220.800781,10478.089844,3849.280029,2163.790039,15241.209961,103.839996,1.062925,...,-13.7,-11.2,-20.0,141.8,150.7,258.012,298.990,301.460,4.4,96.919411


In [73]:
from sklearn.preprocessing import MinMaxScaler


def data_normalisation(df):
    scaler=MinMaxScaler(feature_range=(0,1))
    
    for column_name in list(df.columns.values):
        if column_name == 'Date':
            continue
        the_list = df[column_name].tolist()
        the_list = np.array(the_list)
        the_list = scaler.fit_transform(the_list.reshape(-1,1))
        df.drop(column_name, axis = 1, inplace = True)
        df[column_name] = the_list
    return df
        
def split_train_test_data(df, lookback,num_feature=1):
    df_list = df.loc[:, df.columns != 'Date'].values.tolist()
    res_list = df['Close'].tolist()
    res_list = np.array(res_list)
    print(len(res_list))
    data = []
    res_data = []

    for index in range(len(df_list) - lookback -1): 
        the_list = df_list[index: index + lookback]
        the_list= np.array(the_list)
        the_list = the_list.reshape(-1)
        print(the_list.shape)
        if index==0:
            print(the_list)
        data.append(the_list)
        res_data.append(res_list[index + lookback:index + lookback + 1])

    data = np.array(data)
    res_data = np.array(res_data)
    
    test_set_size = int(np.round(0.2*data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)*2
    
    x_train = data[:train_set_size]
    y_train = res_data[:train_set_size]
    y_train = np.squeeze(y_train)
    
    x_val = data[train_set_size:train_set_size+test_set_size]
    y_val = res_data[train_set_size:train_set_size+test_set_size]
    y_val = np.squeeze(y_val)
    
    x_test = data[train_set_size+test_set_size:]
    y_test = res_data[train_set_size+test_set_size:]
    y_test = np.squeeze(y_test)
    
    return [x_train, y_train,x_val,y_val, x_test, y_test]

In [74]:
lookback = 5 # choose sequence length
normalised_AAPL_df = data_normalisation(AAPL_df)
x_train, y_train,x_val,y_val, x_test, y_test = split_train_test_data(normalised_AAPL_df, lookback,num_feature=1)# change the num_feature value for using more features
# x_test = np.squeeze(x_test)
print(x_train[0])

2007
(170,)
[0.00000000e+00 2.97789024e-02 1.02784539e-01 3.90116018e-02
 7.72102886e-02 3.29682721e-02 2.39441357e-01 9.75707318e-02
 8.55654839e-01 9.40786411e-01 1.35105384e-01 2.26405498e-01
 5.59846283e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 8.96335649e-02 4.20327733e-01 4.25297245e-02
 8.94387577e-01 9.35797665e-01 0.00000000e+00 9.11133925e-01
 6.97565543e-01 7.48349835e-01 6.26110124e-01 2.81150160e-01
 1.64609053e-01 1.08134329e-01 0.00000000e+00 0.00000000e+00
 1.21212121e-01 9.10897164e-01 4.98504487e-04 2.49490421e-02
 8.71105478e-02 3.27150409e-02 6.45328674e-02 2.55621945e-02
 2.13411699e-01 1.09326036e-01 8.06584906e-01 8.83625757e-01
 1.52992888e-01 2.51599441e-01 5.43420338e-01 1.44180580e-01
 5.36026374e-01 4.79061431e-02 1.02939905e-01 8.96335649e-02
 4.20327733e-01 4.25297245e-02 8.94387577e-01 9.35797665e-01
 0.00000000e+00 9.11133925e-01 6.97565543e-01 7.48349835e-01
 6.26110124e-01 2.81150160e-01 1.64609053e-01 1.08134329e-01
 0.00000000e

(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
(170,)
[0.00000000e+00 2.97789024e-02 1.02784539e-01 3.90116018e-02
 7.72102886e-02 3.29682721e-02 2.39441357e-01 9.75707318e-02
 8.55654839e-01 9.40786411e-01 1.35105384e-01 2.26405498e-01
 5.59846283e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 8.96335649e-02 4.20327733e-01 4.25297245e-02
 8.94387577e-01 9.35797665e-01 0.00000000e+00 9.11133925e-01
 6.97565543e-01 7.48349835e-01 6.26110124e-01 2.81150160e-01
 1.64609053e-01 1.08134329e-01 0.00000000e+00 0.00000000e+00
 1.21212121e-01 9.10897164e-01 4.98504487e-04 2.49490421e-02
 8.71105478e-02 3.271504

In [72]:
lookback = 5 # choose sequence length
normalised_AAPL_df = data_normalisation(AAPL_df)
x_train, y_train,x_val,y_val, x_test, y_test = split_train_test_data(normalised_AAPL_df, lookback,num_feature=1)# change the num_feature value for using more features
# x_test = np.squeeze(x_test)
print(x_train[0])

2007
(5, 34)
[[0.00000000e+00 2.97789024e-02 1.02784539e-01 3.90116018e-02
  7.72102886e-02 3.29682721e-02 2.39441357e-01 9.75707318e-02
  8.55654839e-01 9.40786411e-01 1.35105384e-01 2.26405498e-01
  5.59846283e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 8.96335649e-02 4.20327733e-01 4.25297245e-02
  8.94387577e-01 9.35797665e-01 0.00000000e+00 9.11133925e-01
  6.97565543e-01 7.48349835e-01 6.26110124e-01 2.81150160e-01
  1.64609053e-01 1.08134329e-01 0.00000000e+00 0.00000000e+00
  1.21212121e-01 9.10897164e-01]
 [4.98504487e-04 2.49490421e-02 8.71105478e-02 3.27150409e-02
  6.45328674e-02 2.55621945e-02 2.13411699e-01 1.09326036e-01
  8.06584906e-01 8.83625757e-01 1.52992888e-01 2.51599441e-01
  5.43420338e-01 1.44180580e-01 5.36026374e-01 4.79061431e-02
  1.02939905e-01 8.96335649e-02 4.20327733e-01 4.25297245e-02
  8.94387577e-01 9.35797665e-01 0.00000000e+00 9.11133925e-01
  6.97565543e-01 7.48349835e-01 6.26110124e-01 2.81150160e-01
  1.64609053e-01 1.08134

(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
(5, 34)
[[0.00000000e+00 2.97789024e-02 1.02784539e-01 3.90116018e-02
  7.72102886e-02 3.29682721e-02 2.39441357e-01 9.75707318e-02
  8.55654839e-01 9.40786411e-01 1.35105384e-01 2.26405498e-01
  5.59846283e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 8.96335649e-02 4.20327733e-01 4.25297245e-02
  8.94387577e-01 9.35797665e-01 0.00000000e+00 9.11133925e-01
  6.97565543e-01 7.48349835e-01 6.26110124e-01 2.81150160e-01
  1.64609053e-01 1.08134329e-01 0.00000000e+00 0.00000000e+00
  1.21212121e-01 9.10897164e-01]
 [4.98504487e-04 2.49490421e-02 8.71105478e-02 3.27150409e-02
  6.45328674e-02 2.55621945e-02 2.13411699e-01 1.09326036e-01
  8.06584906e-01 8.83625757e-01 1.52992888e-01 2.51599441e-01
  5.43420338e-01 1.44180580e-01 5.36026374e-01 4.79061431e-02
  1.02939905e-01 8.96335649e-02 4.20327733e-01 4.25297245e-02
  8.94387577e-01 9.357976

In [75]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(x_train, y_train)

RandomForestRegressor()

In [76]:
rf.feature_importances_

array([4.00662656e-04, 3.97408370e-04, 4.77798989e-04, 1.04471953e-04,
       2.43305343e-04, 2.18381334e-04, 4.88744045e-05, 1.44889056e-04,
       8.08480842e-05, 5.42533166e-05, 3.71641315e-05, 6.51511399e-05,
       7.27079480e-05, 7.20227501e-05, 1.87117948e-04, 5.15417034e-05,
       2.73996721e-05, 1.04610982e-05, 2.22663599e-06, 9.57325172e-06,
       1.91704231e-06, 9.44232441e-06, 1.96563569e-06, 1.90449590e-05,
       1.37793471e-05, 1.00348940e-05, 3.23233988e-06, 2.70748122e-05,
       7.88720666e-06, 8.82940369e-06, 2.48685064e-06, 5.61710469e-06,
       1.93462089e-06, 6.25105720e-06, 4.72468075e-04, 6.43998043e-04,
       2.22614458e-04, 3.01686792e-04, 3.50772512e-04, 4.68969170e-04,
       5.37354372e-05, 6.48307870e-05, 3.53687470e-05, 4.61210429e-05,
       2.94028815e-05, 4.02446351e-05, 5.11366487e-05, 6.77671892e-05,
       3.69621013e-04, 2.32203211e-05, 2.12879211e-05, 4.88849680e-06,
       1.05138945e-06, 2.45495341e-06, 3.01314857e-06, 6.65446069e-06,
      